In [ ]:
import scraper, extractor

def scrape():
    # %run -i 'scraper.py' # !python 'scraper.py'
    pass

def extract():
    df = extractor.extract_json('Reggaeton_lyrics.json')
    return df

def wrangle(df):
    import statistics
    df['Translation'] = df['Lyrics'].apply(translate_list)
    df['Sentiment'] = df['Lyrics'].apply(analyze_sentiment)
    df['Sent_mean'] = df['Sentiment'].apply(statistics.mean)
    df['No_stopwords'] = df['Lyrics'].apply(remove_stopwords)
    return df

def analyze(df):
    pass

def visualize_analysis(*args):
    for arg in args:
        print(arg)
        print()

def load(csv_name):
    csv_name.to_csv("Reggaeton_lyrics.csv")
    


def main():
    #scrape() # No hace falta porque con el import scrape ya se ejecuta el archivo
    df = extract()
    transformed_df = wrangle(df)
    load(transformed_df)
    
    
if __name__ == "__main__":
    main()